<a href="https://colab.research.google.com/github/Farhad-irp/ML-0-project/blob/main/hw2_iranpak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -U transformers datasets accelerate peft

import torch
import torch.nn as nn
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    set_seed,
)
from peft import LoraConfig, get_peft_model


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.1 MB/s eta 0:00:00


In [2]:
set_seed(42)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "roberta-base"

BATCH_SIZE = 8
MAX_LEN = 128


In [3]:
ds_sst2 = load_dataset("glue", "sst2")
ds_mrpc = load_dataset("glue", "mrpc")

print(ds_sst2)
print(ds_mrpc)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})
DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tok_sst2(examples):
    return tokenizer(examples["sentence"], truncation=True, max_length=MAX_LEN)

def tok_mrpc(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, max_length=MAX_LEN)

sst2_tok = ds_sst2.map(tok_sst2, batched=True)
mrpc_tok = ds_mrpc.map(tok_mrpc, batched=True)

sst2_tok = sst2_tok.remove_columns([c for c in sst2_tok["train"].column_names if c not in ["input_ids","attention_mask","label"]])
mrpc_tok = mrpc_tok.remove_columns([c for c in mrpc_tok["train"].column_names if c not in ["input_ids","attention_mask","label"]])

print("Tokenizer vocab size:", tokenizer.vocab_size)
print("SST-2 columns:", sst2_tok["train"].column_names)
print("MRPC columns:", mrpc_tok["train"].column_names)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Tokenizer vocab size: 50265
SST-2 columns: ['label', 'input_ids', 'attention_mask']
MRPC columns: ['label', 'input_ids', 'attention_mask']


In [5]:
sst2_loader_small = DataLoader(
    sst2_tok["train"].select(range(64)),
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=data_collator
)
batch_sst2 = next(iter(sst2_loader_small))
batch_sst2 = {k: v.to(DEVICE) for k, v in batch_sst2.items()}

mrpc_loader_small = DataLoader(
    mrpc_tok["train"].select(range(64)),
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=data_collator
)
batch_mrpc = next(iter(mrpc_loader_small))
batch_mrpc = {k: v.to(DEVICE) for k, v in batch_mrpc.items()}

print(batch_sst2.keys(), batch_sst2["input_ids"].shape)
print(batch_mrpc.keys(), batch_mrpc["input_ids"].shape)


dict_keys(['input_ids', 'attention_mask', 'labels']) torch.Size([8, 33])
dict_keys(['input_ids', 'attention_mask', 'labels']) torch.Size([8, 67])


In [6]:
model_base_sst2 = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(DEVICE)
model_base_mrpc = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(DEVICE)

with torch.no_grad():
    out1 = model_base_sst2(**batch_sst2)
    out2 = model_base_mrpc(**batch_mrpc)

print("SST-2 logits shape:", out1.logits.shape)
print("MRPC logits shape:", out2.logits.shape)
print("SST-2 base model head:", model_base_sst2.classifier)
print("MRPC base model head:", model_base_mrpc.classifier)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SST-2 logits shape: torch.Size([8, 2])
MRPC logits shape: torch.Size([8, 2])
SST-2 base model head: RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=2, bias=True)
)
MRPC base model head: RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=2, bias=True)
)


In [7]:
def list_qv_modules(model):
    names = []
    for name, _ in model.named_modules():
        if name.endswith(".attention.self.query") or name.endswith(".attention.self.value"):
            names.append(name)
    return names

qv = list_qv_modules(model_base_sst2)
print("Count(query/value modules):", len(qv))
print("First 20 examples:")
for x in qv[:20]:
    print(" -", x)


Count(query/value modules): 24
First 20 examples:
 - roberta.encoder.layer.0.attention.self.query
 - roberta.encoder.layer.0.attention.self.value
 - roberta.encoder.layer.1.attention.self.query
 - roberta.encoder.layer.1.attention.self.value
 - roberta.encoder.layer.2.attention.self.query
 - roberta.encoder.layer.2.attention.self.value
 - roberta.encoder.layer.3.attention.self.query
 - roberta.encoder.layer.3.attention.self.value
 - roberta.encoder.layer.4.attention.self.query
 - roberta.encoder.layer.4.attention.self.value
 - roberta.encoder.layer.5.attention.self.query
 - roberta.encoder.layer.5.attention.self.value
 - roberta.encoder.layer.6.attention.self.query
 - roberta.encoder.layer.6.attention.self.value
 - roberta.encoder.layer.7.attention.self.query
 - roberta.encoder.layer.7.attention.self.value
 - roberta.encoder.layer.8.attention.self.query
 - roberta.encoder.layer.8.attention.self.value
 - roberta.encoder.layer.9.attention.self.query
 - roberta.encoder.layer.9.attention.s

In [8]:
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.0,
    bias="none",
    target_modules=["query", "value"],
    task_type="SEQ_CLS"
)

model_lora_sst2 = get_peft_model(
    AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2),
    lora_cfg
).to(DEVICE)

total = sum(p.numel() for p in model_lora_sst2.parameters())
trainable = sum(p.numel() for p in model_lora_sst2.parameters() if p.requires_grad)
print(f"Total params:     {total:,}")
print(f"Trainable params: {trainable:,}")
print(f"Trainable ratio:  {trainable/total*100:.4f}%")

with torch.no_grad():
    out = model_lora_sst2(**batch_sst2)
print("LoRA model logits shape:", out.logits.shape)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total params:     125,534,212
Trainable params: 887,042
Trainable ratio:  0.7066%
LoRA model logits shape: torch.Size([8, 2])


In [9]:
class SimpleAdapter(nn.Module):
    def __init__(self, hidden_size=768, bottleneck=64):
        super().__init__()
        self.down = nn.Linear(hidden_size, bottleneck)
        self.act = nn.ReLU()
        self.up = nn.Linear(bottleneck, hidden_size)

    def forward(self, x):
        return self.up(self.act(self.down(x)))

def add_adapters_to_roberta_stable(model, bottleneck=64):

    for i, layer in enumerate(model.roberta.encoder.layer):
        layer.attn_adapter = SimpleAdapter(model.config.hidden_size, bottleneck)

        old_layer_forward = layer.forward  # bound method

        def wrapped_layer_forward(
            self_layer,
            hidden_states,
            attention_mask=None,
            head_mask=None,
            encoder_hidden_states=None,
            encoder_attention_mask=None,
            past_key_value=None,
            output_attentions=False,
            cache_position=None,
            **kwargs
        ):
            outputs = old_layer_forward(
                hidden_states,
                attention_mask=attention_mask,
                head_mask=head_mask,
                encoder_hidden_states=encoder_hidden_states,
                encoder_attention_mask=encoder_attention_mask,
                past_key_value=past_key_value,
                output_attentions=output_attentions,
                cache_position=cache_position,
                **kwargs
            )

            hs = outputs[0]
            hs = hs + self_layer.attn_adapter(hs)

            return (hs,) + outputs[1:]

        layer.forward = wrapped_layer_forward.__get__(layer, type(layer))

    return model


In [10]:
model_adapter_sst2 = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(DEVICE)
model_adapter_sst2 = add_adapters_to_roberta_stable(model_adapter_sst2, bottleneck=64).to(DEVICE)

# freeze everything
for p in model_adapter_sst2.parameters():
    p.requires_grad = False

# unfreeze adapters + classifier
for layer in model_adapter_sst2.roberta.encoder.layer:
    for p in layer.attn_adapter.parameters():
        p.requires_grad = True

for p in model_adapter_sst2.classifier.parameters():
    p.requires_grad = True

total = sum(p.numel() for p in model_adapter_sst2.parameters())
trainable = sum(p.numel() for p in model_adapter_sst2.parameters() if p.requires_grad)
print(f"Total params:     {total:,}")
print(f"Trainable params: {trainable:,}")
print(f"Trainable ratio:  {trainable/total*100:.4f}%")

with torch.no_grad():
    out = model_adapter_sst2(**batch_sst2)
print("Adapter model logits shape:", out.logits.shape)

trainable_names = [n for n, p in model_adapter_sst2.named_parameters() if p.requires_grad]
print("Trainable parameter name samples:")
for n in trainable_names[:10]:
    print(" -", n)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total params:     125,836,802
Trainable params: 1,781,762
Trainable ratio:  1.4159%
Adapter model logits shape: torch.Size([8, 2])
Trainable parameter name samples:
 - roberta.encoder.layer.0.attn_adapter.down.weight
 - roberta.encoder.layer.0.attn_adapter.down.bias
 - roberta.encoder.layer.0.attn_adapter.up.weight
 - roberta.encoder.layer.0.attn_adapter.up.bias
 - roberta.encoder.layer.1.attn_adapter.down.weight
 - roberta.encoder.layer.1.attn_adapter.down.bias
 - roberta.encoder.layer.1.attn_adapter.up.weight
 - roberta.encoder.layer.1.attn_adapter.up.bias
 - roberta.encoder.layer.2.attn_adapter.down.weight
 - roberta.encoder.layer.2.attn_adapter.down.bias


/tmp/ipython-input-3983512676.py:30: FutureWarning: Both `past_key_value` and `past_key_values` are set for `RobertaLayer.forward`. Using `past_key_values=None` and ignoring deprecated `past_key_value=None`.
  outputs = old_layer_forward(


In [11]:
class PrefixPromptEncoder(nn.Module):
    def __init__(self, prefix_len=10, hidden_size=768, num_heads=12, dropout=0.0):
        super().__init__()
        self.prefix_len = prefix_len
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        self.embedding = nn.Embedding(prefix_len, hidden_size)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, 2 * hidden_size)  # K and V
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, batch_size, device):
        idx = torch.arange(self.prefix_len, device=device)
        e = self.embedding(idx)                  # [P,H]
        proj = self.dropout(self.mlp(e))         # [P,2H]
        pk, pv = proj.split(self.hidden_size, dim=-1)

        pk = pk.view(self.prefix_len, self.num_heads, self.head_dim).permute(1, 0, 2)
        pv = pv.view(self.prefix_len, self.num_heads, self.head_dim).permute(1, 0, 2)

        pk = pk.unsqueeze(0).expand(batch_size, -1, -1, -1).contiguous()
        pv = pv.unsqueeze(0).expand(batch_size, -1, -1, -1).contiguous()
        return pk, pv


def apply_prefix_tuning_v3(model, prefix_len=10, dropout=0.0):
    cfg = model.config
    L = cfg.num_hidden_layers
    H = cfg.hidden_size
    nH = cfg.num_attention_heads
    dH = H // nH

    model.prefix_len = prefix_len
    model.prefix_encoders = nn.ModuleList([
        PrefixPromptEncoder(prefix_len, H, nH, dropout) for _ in range(L)
    ])

    for layer_idx, layer in enumerate(model.roberta.encoder.layer):
        attn = layer.attention.self

        def wrapped_forward(
            self,
            hidden_states,
            attention_mask=None,
            head_mask=None,
            encoder_hidden_states=None,
            encoder_attention_mask=None,
            past_key_value=None,
            past_key_values=None,
            output_attentions=False,
            cache_position=None,
            use_cache=None,
            **kwargs
        ):
            bsz, seqlen, _ = hidden_states.shape
            device = hidden_states.device

            q = self.query(hidden_states)
            k = self.key(hidden_states)
            v = self.value(hidden_states)

            q = q.view(bsz, seqlen, nH, dH).permute(0, 2, 1, 3)
            k = k.view(bsz, seqlen, nH, dH).permute(0, 2, 1, 3)
            v = v.view(bsz, seqlen, nH, dH).permute(0, 2, 1, 3)

            pk, pv = model.prefix_encoders[layer_idx](batch_size=bsz, device=device)
            k = torch.cat([pk, k], dim=2)
            v = torch.cat([pv, v], dim=2)

            scores = torch.matmul(q, k.transpose(-1, -2)) / (dH ** 0.5)

            if attention_mask is not None:
                # attention_mask shape: [B,1,1,seq]
                prefix_mask = torch.zeros(
                    attention_mask.size(0), attention_mask.size(1), attention_mask.size(2), prefix_len,
                    device=attention_mask.device, dtype=attention_mask.dtype
                )
                scores = scores + torch.cat([prefix_mask, attention_mask], dim=-1)

            probs = torch.softmax(scores, dim=-1)
            probs = self.dropout(probs)

            if head_mask is not None:
                probs = probs * head_mask

            context = torch.matmul(probs, v)  # [B,heads,seq,head_dim]
            context = context.permute(0, 2, 1, 3).contiguous().view(bsz, seqlen, H)

            return (context, probs) if output_attentions else (context,)

        attn.forward = wrapped_forward.__get__(attn, type(attn))

    return model


def freeze_backbone_except_prefix_and_head(model):
    for p in model.parameters():
        p.requires_grad = False
    for p in model.prefix_encoders.parameters():
        p.requires_grad = True
    for p in model.classifier.parameters():
        p.requires_grad = True


In [12]:
# SST-2
model_prefix_sst2 = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
).to(DEVICE)

# Prefix-Tuning v3
model_prefix_sst2 = apply_prefix_tuning_v3(
    model_prefix_sst2,
    prefix_len=10,
    dropout=0.0
).to(DEVICE)

freeze_backbone_except_prefix_and_head(model_prefix_sst2)

total = sum(p.numel() for p in model_prefix_sst2.parameters())
trainable = sum(p.numel() for p in model_prefix_sst2.parameters() if p.requires_grad)
print(f"Total params:     {total:,}")
print(f"Trainable params: {trainable:,}")
print(f"Trainable ratio:  {trainable/total*100:.4f}%")

with torch.no_grad():
    out = model_prefix_sst2(**batch_sst2)

print("Prefix(v3) logits shape:", out.logits.shape)

trainable_names = [n for n, p in model_prefix_sst2.named_parameters() if p.requires_grad]
print("Trainable parameter name samples:")
for n in trainable_names[:10]:
    print(" -", n)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total params:     146,000,642
Trainable params: 21,945,602
Trainable ratio:  15.0312%
Prefix(v3) logits shape: torch.Size([8, 2])
Trainable parameter name samples:
 - classifier.dense.weight
 - classifier.dense.bias
 - classifier.out_proj.weight
 - classifier.out_proj.bias
 - prefix_encoders.0.embedding.weight
 - prefix_encoders.0.mlp.0.weight
 - prefix_encoders.0.mlp.0.bias
 - prefix_encoders.0.mlp.2.weight
 - prefix_encoders.0.mlp.2.bias
 - prefix_encoders.1.embedding.weight


In [13]:
with torch.no_grad():
    out_lora = model_lora_sst2(
        **batch_sst2,
        output_hidden_states=True,
        return_dict=True
    )
    out_adapter = model_adapter_sst2(
        **batch_sst2,
        output_hidden_states=True,
        return_dict=True
    )
    out_prefix = model_prefix_sst2(
        **batch_sst2,
        output_hidden_states=True,
        return_dict=True
    )

print("LoRA:   ", len(out_lora.hidden_states), out_lora.hidden_states[-1].shape)
print("Adapter:", len(out_adapter.hidden_states), out_adapter.hidden_states[-1].shape)
print("Prefix: ", len(out_prefix.hidden_states), out_prefix.hidden_states[-1].shape)


LoRA:    13 torch.Size([8, 33, 768])
Adapter: 13 torch.Size([8, 33, 768])
Prefix:  13 torch.Size([8, 33, 768])


/tmp/ipython-input-3983512676.py:30: FutureWarning: Both `past_key_value` and `past_key_values` are set for `RobertaLayer.forward`. Using `past_key_values=None` and ignoring deprecated `past_key_value=None`.
  outputs = old_layer_forward(


In [14]:
class LayerGate(nn.Module):
    def __init__(self, num_layers=12, num_methods=3):
        super().__init__()
        self.w = nn.Parameter(torch.zeros(num_layers, num_methods))  # sigmoid -> 0.5

    def raw_gates(self):
        return torch.sigmoid(self.w)  # [12,3]

    def norm_gates(self):
        g = self.raw_gates()
        return g / (g.sum(dim=1, keepdim=True) + 1e-8)


class LayerGatedEnsembleEntropy(nn.Module):
    def __init__(self, model_lora, model_adapter, model_prefix, num_layers=12, lambda_sparse=0.0):
        super().__init__()
        self.model_lora = model_lora
        self.model_adapter = model_adapter
        self.model_prefix = model_prefix

        self.gate = LayerGate(num_layers=num_layers, num_methods=3)
        self.lambda_sparse = lambda_sparse

        self.classifier = model_lora.classifier

    def entropy_sparse_loss(self, g_norm):
        if self.lambda_sparse <= 0:
            return None
        ent = -(g_norm * torch.log(g_norm + 1e-8)).sum(dim=1).mean()
        return self.lambda_sparse * ent

    def forward(self, input_ids, attention_mask=None, labels=None):
        batch = {"input_ids": input_ids, "attention_mask": attention_mask}


In [15]:
for m in [model_lora_sst2, model_adapter_sst2, model_prefix_sst2]:
    for p in m.parameters():
        p.requires_grad = False

layer_ens = LayerGatedEnsembleEntropy(
    model_lora=model_lora_sst2,
    model_adapter=model_adapter_sst2,
    model_prefix=model_prefix_sst2,
    num_layers=12,
    lambda_sparse=0.0
).to(DEVICE)

for p in layer_ens.parameters():
    p.requires_grad = False
for p in layer_ens.gate.parameters():
    p.requires_grad = True

trainable = [(n, p.numel()) for n, p in layer_ens.named_parameters() if p.requires_grad]
print("Trainable:", trainable)
print("Total trainable:", sum(c for _, c in trainable))


Trainable: [('gate.w', 36)]
Total trainable: 36


In [16]:

class LayerGate(nn.Module):
    def __init__(self, num_layers=12, num_methods=3):
        super().__init__()
        self.w = nn.Parameter(torch.zeros(num_layers, num_methods))  # sigmoid -> 0.5

    def raw_gates(self):
        return torch.sigmoid(self.w)  # [12,3]

    def norm_gates(self):
        g = self.raw_gates()
        return g / (g.sum(dim=1, keepdim=True) + 1e-8)


class LayerGatedEnsembleEntropy(nn.Module):
    def __init__(self, model_lora, model_adapter, model_prefix, num_layers=12, lambda_sparse=0.0):
        super().__init__()
        self.model_lora = model_lora
        self.model_adapter = model_adapter
        self.model_prefix = model_prefix

        self.gate = LayerGate(num_layers=num_layers, num_methods=3)
        self.lambda_sparse = lambda_sparse

        self.classifier = model_lora.classifier

    def entropy_sparse_loss(self, g_norm):
        if self.lambda_sparse <= 0:
            return None
        ent = -(g_norm * torch.log(g_norm + 1e-8)).sum(dim=1).mean()
        return self.lambda_sparse * ent

    def forward(self, input_ids, attention_mask=None, labels=None):
        batch = {"input_ids": input_ids, "attention_mask": attention_mask}

        out_l = self.model_lora(**batch, output_hidden_states=True, return_dict=True)
        out_a = self.model_adapter(**batch, output_hidden_states=True, return_dict=True)
        out_p = self.model_prefix(**batch, output_hidden_states=True, return_dict=True)

        hs_l, hs_a, hs_p = out_l.hidden_states, out_a.hidden_states, out_p.hidden_states

        g_raw = self.gate.raw_gates()
        g = self.gate.norm_gates()

        h = hs_l[0]
        for layer_idx in range(1, 13):
            gl, ga, gp = g[layer_idx - 1]
            h = gl * hs_l[layer_idx] + ga * hs_a[layer_idx] + gp * hs_p[layer_idx]

        logits = self.classifier(h)

        loss = None
        if labels is not None:
            ce = nn.CrossEntropyLoss()(logits.view(-1, logits.size(-1)), labels.view(-1))
            sp = self.entropy_sparse_loss(g)
            loss = ce if sp is None else (ce + sp)

        return {"loss": loss, "logits": logits, "g_raw": g_raw.detach(), "g": g.detach()}



for m in [model_lora_sst2, model_adapter_sst2, model_prefix_sst2]:
    for p in m.parameters():
        p.requires_grad = False

layer_ens = LayerGatedEnsembleEntropy(
    model_lora=model_lora_sst2,
    model_adapter=model_adapter_sst2,
    model_prefix=model_prefix_sst2,
    num_layers=12,
    lambda_sparse=0.0
).to(DEVICE)

for p in layer_ens.parameters():
    p.requires_grad = False
for p in layer_ens.gate.parameters():
    p.requires_grad = True

print(" Trainable params:",
      sum(p.numel() for p in layer_ens.parameters() if p.requires_grad))


 Trainable params: 36


In [17]:
y = batch_sst2["label"] if "label" in batch_sst2 else batch_sst2["labels"]

with torch.no_grad():
    out = layer_ens(
        input_ids=batch_sst2["input_ids"],
        attention_mask=batch_sst2.get("attention_mask", None),
        labels=y
    )

print("Layer-ensemble logits shape:", out["logits"].shape)
print("g_raw shape:", out["g_raw"].shape)
print("g_norm shape:", out["g"].shape)
print("First layer raw gates:", out["g_raw"][0].cpu().numpy())
print("First layer norm gates:", out["g"][0].cpu().numpy())


Layer-ensemble logits shape: torch.Size([8, 2])
g_raw shape: torch.Size([12, 3])
g_norm shape: torch.Size([12, 3])
First layer raw gates: [0.5 0.5 0.5]
First layer norm gates: [0.33333334 0.33333334 0.33333334]


/tmp/ipython-input-3983512676.py:30: FutureWarning: Both `past_key_value` and `past_key_values` are set for `RobertaLayer.forward`. Using `past_key_values=None` and ignoring deprecated `past_key_value=None`.
  outputs = old_layer_forward(


In [18]:
train_loader = DataLoader(
    sst2_tok["train"].select(range(512)),
    batch_size=16,
    shuffle=True,
    collate_fn=data_collator
)

def show_layers(ens, layers=(0, 11)):
    g_raw = ens.gate.raw_gates().detach().cpu()
    g = ens.gate.norm_gates().detach().cpu()
    for L in layers:
        print(f"Layer {L:2d} raw :", g_raw[L].numpy(), " norm:", g[L].numpy())


layer_ens.lambda_sparse = 0.0
opt = torch.optim.AdamW(layer_ens.gate.parameters(), lr=1e-1)

print("Before CE-only training:")
show_layers(layer_ens)

layer_ens.train()
for step, batch in enumerate(train_loader):
    if step >= 30:
        break
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    y = batch["label"] if "label" in batch else batch["labels"]

    opt.zero_grad()
    out = layer_ens(batch["input_ids"], batch.get("attention_mask", None), y)
    out["loss"].backward()
    opt.step()

    if step % 10 == 0:
        print(f"step={step:02d} loss={out['loss'].item():.4f}")

print("After CE-only training:")
show_layers(layer_ens)

layer_ens.lambda_sparse = 0.2
opt = torch.optim.AdamW(layer_ens.gate.parameters(), lr=1e-1)

print("Before CE+Entropy training:")
show_layers(layer_ens)

for step, batch in enumerate(train_loader):
    if step >= 80:
        break
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    y = batch["label"] if "label" in batch else batch["labels"]

    opt.zero_grad()
    out = layer_ens(batch["input_ids"], batch.get("attention_mask", None), y)
    out["loss"].backward()
    opt.step()

    if step % 20 == 0:
        print(f"step={step:02d} loss={out['loss'].item():.4f}")

print("After CE+Entropy training:")
show_layers(layer_ens)


Before CE-only training:
Layer  0 raw : [0.5 0.5 0.5]  norm: [0.33333334 0.33333334 0.33333334]
Layer 11 raw : [0.5 0.5 0.5]  norm: [0.33333334 0.33333334 0.33333334]


/tmp/ipython-input-3983512676.py:30: FutureWarning: Both `past_key_value` and `past_key_values` are set for `RobertaLayer.forward`. Using `past_key_values=None` and ignoring deprecated `past_key_value=None`.
  outputs = old_layer_forward(


step=00 loss=0.6907
step=10 loss=0.7144
step=20 loss=0.6979
After CE-only training:
Layer  0 raw : [0.5 0.5 0.5]  norm: [0.33333334 0.33333334 0.33333334]
Layer 11 raw : [0.39918905 0.54121685 0.5344785 ]  norm: [0.27065787 0.36695543 0.3623867 ]
Before CE+Entropy training:
Layer  0 raw : [0.5 0.5 0.5]  norm: [0.33333334 0.33333334 0.33333334]
Layer 11 raw : [0.39918905 0.54121685 0.5344785 ]  norm: [0.27065787 0.36695543 0.3623867 ]
step=00 loss=0.9272
step=20 loss=0.9112
After CE+Entropy training:
Layer  0 raw : [0.50039697 0.50039697 0.50039697]  norm: [0.33333334 0.33333334 0.33333334]
Layer 11 raw : [0.28651914 0.154075   0.9096739 ]  norm: [0.21219425 0.11410696 0.6736987 ]


In [19]:
import numpy as np

g_raw = layer_ens.gate.raw_gates().detach().cpu().numpy()   # [12,3]
g = layer_ens.gate.norm_gates().detach().cpu().numpy()      # [12,3]

methods = ["LoRA", "Adapter", "Prefix"]

print("Final layer-wise gates (normalized):")
for i in range(12):
    top = int(np.argmax(g[i]))
    print(f"Layer {i:02d}: {g[i]}   top={methods[top]}")

print("\nMean gate weight across layers:", g.mean(axis=0), " -> ", dict(zip(methods, g.mean(axis=0))))
print("Top method counts:", {m: int((np.argmax(g, axis=1) == j).sum()) for j, m in enumerate(methods)})


Final layer-wise gates (normalized):
Layer 00: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 01: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 02: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 03: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 04: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 05: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 06: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 07: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 08: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 09: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 10: [0.33333334 0.33333334 0.33333334]   top=LoRA
Layer 11: [0.21219425 0.11410696 0.6736987 ]   top=Prefix

Mean gate weight across layers: [0.32323837 0.31506443 0.36169708]  ->  {'LoRA': np.float32(0.32323837), 'Adapter': np.float32(0.31506443), 'Prefix': np.float32(0.36169708)}
Top method counts: {'LoRA': 11, 'Adapter': 0, 'Prefix': 1}


In [20]:
for L in [0, 5, 11]:
    print(f"Layer {L:02d} raw : {g_raw[L]}   norm: {g[L]}")


Layer 00 raw : [0.50039697 0.50039697 0.50039697]   norm: [0.33333334 0.33333334 0.33333334]
Layer 05 raw : [0.50039697 0.50039697 0.50039697]   norm: [0.33333334 0.33333334 0.33333334]
Layer 11 raw : [0.28651914 0.154075   0.9096739 ]   norm: [0.21219425 0.11410696 0.6736987 ]


In [21]:
class InstanceGate(nn.Module):
    def __init__(self, hidden_size=768, num_layers=12, num_methods=3, bottleneck=128):
        super().__init__()
        self.num_layers = num_layers
        self.num_methods = num_methods

        self.net = nn.Sequential(
            nn.Linear(hidden_size, bottleneck),
            nn.GELU(),
            nn.Linear(bottleneck, num_layers * num_methods)
        )

    def forward(self, x_cls):
        # x_cls: [B, H]
        logits = self.net(x_cls).view(-1, self.num_layers, self.num_methods)  # [B,12,3]
        gates = torch.softmax(logits, dim=-1)
        return logits, gates


class InstanceGatedEnsemble(nn.Module):
    def __init__(self, model_lora, model_adapter, model_prefix, num_layers=12, lambda_entropy=0.0):
        super().__init__()
        self.model_lora = model_lora
        self.model_adapter = model_adapter
        self.model_prefix = model_prefix
        self.gate = InstanceGate(hidden_size=768, num_layers=num_layers, num_methods=3, bottleneck=128)
        self.lambda_entropy = lambda_entropy
        self.classifier = model_lora.classifier  # reuse head

    def entropy_penalty(self, gates):
        if self.lambda_entropy <= 0:
            return None
        ent = -(gates * torch.log(gates + 1e-8)).sum(dim=-1).mean()  # mean over B and layers
        return self.lambda_entropy * ent

    def forward(self, input_ids, attention_mask=None, labels=None):
        batch = {"input_ids": input_ids, "attention_mask": attention_mask}

        out_l = self.model_lora(**batch, output_hidden_states=True, return_dict=True)
        out_a = self.model_adapter(**batch, output_hidden_states=True, return_dict=True)
        out_p = self.model_prefix(**batch, output_hidden_states=True, return_dict=True)

        hs_l, hs_a, hs_p = out_l.hidden_states, out_a.hidden_states, out_p.hidden_states

        x_cls = hs_l[-1][:, 0, :]  # [B,768]
        logits_gate, gates = self.gate(x_cls)  # [B,12,3] each

        h = hs_l[0]
        for layer_idx in range(1, 13):
            g = gates[:, layer_idx - 1, :]  # [B,3]
            gl = g[:, 0].view(-1, 1, 1)
            ga = g[:, 1].view(-1, 1, 1)
            gp = g[:, 2].view(-1, 1, 1)
            h = gl * hs_l[layer_idx] + ga * hs_a[layer_idx] + gp * hs_p[layer_idx]

        logits = self.classifier(h)

        loss = None
        if labels is not None:
            ce = nn.CrossEntropyLoss()(logits.view(-1, logits.size(-1)), labels.view(-1))
            ep = self.entropy_penalty(gates)
            loss = ce if ep is None else (ce + ep)

        return {"loss": loss, "logits": logits, "gates": gates.detach(), "gate_logits": logits_gate.detach()}


In [22]:
for m in [model_lora_sst2, model_adapter_sst2, model_prefix_sst2]:
    for p in m.parameters():
        p.requires_grad = False

instance_ens = InstanceGatedEnsemble(
    model_lora=model_lora_sst2,
    model_adapter=model_adapter_sst2,
    model_prefix=model_prefix_sst2,
    num_layers=12,
    lambda_entropy=0.0
).to(DEVICE)

for p in instance_ens.parameters():
    p.requires_grad = False
for p in instance_ens.gate.parameters():
    p.requires_grad = True

trainable = [(n, p.numel()) for n, p in instance_ens.named_parameters() if p.requires_grad]
print("Trainable:", trainable[:10])
print("Total trainable:", sum(c for _, c in trainable))

y = batch_sst2["label"] if "label" in batch_sst2 else batch_sst2["labels"]
with torch.no_grad():
    out = instance_ens(
        input_ids=batch_sst2["input_ids"],
        attention_mask=batch_sst2.get("attention_mask", None),
        labels=y
    )

print("Instance-ensemble logits shape:", out["logits"].shape)
print("Gates shape:", out["gates"].shape)
print("Sample gates for layer0 (first sample):", out["gates"][0, 0].cpu().numpy())


Trainable: [('gate.net.0.weight', 98304), ('gate.net.0.bias', 128), ('gate.net.2.weight', 4608), ('gate.net.2.bias', 36)]
Total trainable: 103076
Instance-ensemble logits shape: torch.Size([8, 2])
Gates shape: torch.Size([8, 12, 3])
Sample gates for layer0 (first sample): [0.3380861  0.31372368 0.3481902 ]


/tmp/ipython-input-3983512676.py:30: FutureWarning: Both `past_key_value` and `past_key_values` are set for `RobertaLayer.forward`. Using `past_key_values=None` and ignoring deprecated `past_key_value=None`.
  outputs = old_layer_forward(


In [23]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    sst2_tok["train"].select(range(512)),
    batch_size=16,
    shuffle=True,
    collate_fn=data_collator
)

def inspect_instance_gates(ens, layers=(0, 11), n=3):
    with torch.no_grad():
        batch = next(iter(train_loader))
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        y = batch["label"] if "label" in batch else batch["labels"]
        out = ens(batch["input_ids"], batch.get("attention_mask", None), y)
        g = out["gates"].mean(dim=0).cpu().numpy()  # [12,3]
    for L in layers:
        print(f"Layer {L:02d} mean gates:", g[L])

# CE-only
instance_ens.lambda_entropy = 0.0
opt = torch.optim.AdamW(instance_ens.gate.parameters(), lr=5e-4)

print("Before CE-only:")
inspect_instance_gates(instance_ens)

instance_ens.train()
for step, batch in enumerate(train_loader):
    if step >= 200:
        break
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    y = batch["label"] if "label" in batch else batch["labels"]

    opt.zero_grad()
    out = instance_ens(batch["input_ids"], batch.get("attention_mask", None), y)
    out["loss"].backward()
    opt.step()

    if step % 50 == 0:
        print(f"CE step={step:03d} loss={out['loss'].item():.4f}")

print("After CE-only:")
inspect_instance_gates(instance_ens)

# CE + Entropy
instance_ens.lambda_entropy = 0.1
opt = torch.optim.AdamW(instance_ens.gate.parameters(), lr=5e-4)

print("Before CE+Entropy:")
inspect_instance_gates(instance_ens)

for step, batch in enumerate(train_loader):
    if step >= 300:
        break
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    y = batch["label"] if "label" in batch else batch["labels"]

    opt.zero_grad()
    out = instance_ens(batch["input_ids"], batch.get("attention_mask", None), y)
    out["loss"].backward()
    opt.step()

    if step % 100 == 0:
        print(f"CE+Ent step={step:03d} loss={out['loss'].item():.4f}")

print("After CE+Entropy:")
inspect_instance_gates(instance_ens)


Before CE-only:


/tmp/ipython-input-3983512676.py:30: FutureWarning: Both `past_key_value` and `past_key_values` are set for `RobertaLayer.forward`. Using `past_key_values=None` and ignoring deprecated `past_key_value=None`.
  outputs = old_layer_forward(


Layer 00 mean gates: [0.3348004  0.30344468 0.36175492]
Layer 11 mean gates: [0.31791368 0.35116994 0.33091635]
CE step=000 loss=0.7075
After CE-only:
Layer 00 mean gates: [0.33541185 0.30262393 0.3619643 ]
Layer 11 mean gates: [0.41709214 0.16293018 0.41997772]
Before CE+Entropy:
Layer 00 mean gates: [0.33788484 0.30868855 0.35342658]
Layer 11 mean gates: [0.40443414 0.1821016  0.41346425]
CE+Ent step=000 loss=0.7882
After CE+Entropy:
Layer 00 mean gates: [0.06262057 0.0629541  0.8744253 ]
Layer 11 mean gates: [0.33106267 0.0191097  0.64982766]


In [24]:
import numpy as np

instance_ens.eval()
with torch.no_grad():
    batch = next(iter(train_loader))
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    y = batch["label"] if "label" in batch else batch["labels"]
    out = instance_ens(batch["input_ids"], batch.get("attention_mask", None), y)
    g_mean = out["gates"].mean(dim=0).cpu().numpy()  # [12,3]

methods = ["LoRA", "Adapter", "Prefix"]
print("Mean gates per layer (after CE+Entropy) on one batch:")
for L in range(12):
    top = int(np.argmax(g_mean[L]))
    print(f"Layer {L:02d}: {g_mean[L]}  top={methods[top]}")

print("\nMean across layers:", g_mean.mean(axis=0), dict(zip(methods, g_mean.mean(axis=0))))


Mean gates per layer (after CE+Entropy) on one batch:
Layer 00: [0.06601266 0.06702955 0.8669578 ]  top=Prefix
Layer 01: [0.21841885 0.7368791  0.04470196]  top=Adapter
Layer 02: [0.4484861  0.04362932 0.50788456]  top=Prefix
Layer 03: [0.04208414 0.27468485 0.683231  ]  top=Prefix
Layer 04: [0.78913426 0.04535934 0.16550645]  top=LoRA
Layer 05: [0.06033789 0.06991432 0.8697478 ]  top=Prefix
Layer 06: [0.04802656 0.05788345 0.89409006]  top=Prefix
Layer 07: [0.03268089 0.07071753 0.89660156]  top=Prefix
Layer 08: [0.03913514 0.84942186 0.11144294]  top=Adapter
Layer 09: [0.0438797  0.7846743  0.17144598]  top=Adapter
Layer 10: [0.0503555 0.8782405 0.071404 ]  top=Adapter
Layer 11: [0.35888523 0.01885086 0.6222639 ]  top=Prefix

Mean across layers: [0.18311976 0.32477376 0.49210647] {'LoRA': np.float32(0.18311976), 'Adapter': np.float32(0.32477376), 'Prefix': np.float32(0.49210647)}


/tmp/ipython-input-3983512676.py:30: FutureWarning: Both `past_key_value` and `past_key_values` are set for `RobertaLayer.forward`. Using `past_key_values=None` and ignoring deprecated `past_key_value=None`.
  outputs = old_layer_forward(


In [25]:
with torch.no_grad():
    batch = next(iter(train_loader))
    batch = {k: v.to(DEVICE) for k, v in batch.items()}
    y = batch["label"] if "label" in batch else batch["labels"]
    out = instance_ens(batch["input_ids"], batch.get("attention_mask", None), y)
    g = out["gates"].cpu().numpy()  # [B,12,3]

for i in [0, 1, 2]:
    print(f"\nSample {i}:")
    print(" layer00:", g[i, 0])
    print(" layer11:", g[i, 11])


/tmp/ipython-input-3983512676.py:30: FutureWarning: Both `past_key_value` and `past_key_values` are set for `RobertaLayer.forward`. Using `past_key_values=None` and ignoring deprecated `past_key_value=None`.
  outputs = old_layer_forward(



Sample 0:
 layer00: [0.06255364 0.0639426  0.8735038 ]
 layer11: [0.358712   0.01773726 0.6235507 ]

Sample 1:
 layer00: [0.06968336 0.06993845 0.86037827]
 layer11: [0.36038274 0.02008888 0.61952835]

Sample 2:
 layer00: [0.07034992 0.070899   0.8587511 ]
 layer11: [0.35971642 0.02019863 0.620085  ]
